In [5]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI

In [6]:
import gradio as gr

In [7]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [8]:
openai = OpenAI()

In [9]:
directory="hrdataset/employees"
def read_markdown_files(directory):
    """Read and load content from all Markdown files in a directory."""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                documents.append({"filename": filename, "content": f.read()})
    return documents

In [10]:
docs=read_markdown_files(directory)

In [108]:
def build_user_messages(question):
    message = ""
    message += f"Here is info about all the employees {docs}"
    message += f"now answer query {question}"
    return message

In [103]:
system_message="You are a helpful HR assistant"

In [110]:
def process_question_gpt(message, history):
    prompt = build_user_messages(message)

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [118]:
demo = gr.ChatInterface(
    process_question_gpt,
    chatbot=gr.Chatbot(
        height=450,
        placeholder="Ask the question about Employees",
        type="messages",  # Proper rendering of chat format
    ),
    textbox=gr.Textbox(
        placeholder="What do you want to know about your co-workers?",
        container=False,
        scale=7,
    ),
    title="Know your co-workers?",
    examples=["Who is the CEO of the company?", "who joined the company recently"],
    cache_examples=False,
)


/opt/anaconda3/envs/genai/lib/python3.11/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


In [119]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7925
* Running on public URL: https://91f5a00c9b5a4c160d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Example 2 - With Memory 

In [150]:
def build_messages_with_history(question, history):
    prompt = ""
    
    messages = []
    for entry in history:
        if entry["role"] == "user":
            messages.append({"role": "user", "content": entry["content"]})
        elif entry["role"] == "assistant":
            messages.append({"role": "assistant", "content": entry["content"]})
    
    prompt += f"Here is info about all the employees {docs}"
    prompt += f"{messages}"
    prompt += f"now answer query {question}"

    #print(prompt)
    return prompt

In [151]:
def process_question_gpt_with_history(message, history):

    prompt = build_messages_with_history(message, history)

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [154]:
demo2 = gr.ChatInterface(
    process_question_gpt_with_history,
    chatbot=gr.Chatbot(
        height=450,
        placeholder="Ask the question about Employees",
        type="messages",  # Proper rendering of chat format
    ),
    textbox=gr.Textbox(
        placeholder="What do you want to know about your co-workers?",
        container=False,
        scale=7,
    ),
    title="Know your co-workers?",
    examples=["Who is the CEO of the company?", "who joined the company recently"],
    cache_examples=False,
)

/opt/anaconda3/envs/genai/lib/python3.11/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


In [156]:
demo2.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* Running on public URL: https://c1dd51b9bfd67a460e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
